# Digital Twin Fairways
This notebook provides an example on how to use the digital twin backend. It runs a simulation based on fairway information system of the Netherlands. You can define sites (with cargo), climate conditions and ships. Ships will transport the goods from A to B.

In [1]:
import sys
sys.path.append(r"D:\01. Projecten\[130878] DTV vaarwegen\digitaltwin-waterway\dtv_backend")

import datetime

import geojson
import simpy
import time
import json
import shapely
import pandas as pd
import networkx as nx
from networkx.readwrite import json_graph

# library to load the fairway information network
import dtv_backend.fis
# the simpy processes and objects
import dtv_backend.simple
import dtv_backend.network.network_utilities

# reload for debugging purposes
%load_ext autoreload
%autoreload 2

### Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [2]:
# example input
with open('../../dtv_backend/tests/test-berth/config.json') as f:
    config = geojson.load(f)

### Simulation environment
Setup an simulation environment with a time that starts today. You can also choose a date in the past or future. Using the date 0 won't work on windows. 

In [3]:
# Initialize an environment with a real time
now = datetime.datetime.now()
initial_time = now.timestamp()
env = simpy.Environment(initial_time=initial_time)
env.epoch = now

### Network
Here we load the digital twin network. The topological fairway network derived from the Dutch Fairway Information System. The data is processed to be topological connected and usable for transport network analysis.  

In [4]:
## TEMPORARILY LOAD LOCAL FILE ##
def read_json_file(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)

file = r"D:\01. Projecten\[130878] DTV vaarwegen\02. Data\network_digital_twin_v0.3.json"
G = read_json_file(file)

for n in G.nodes:
    G.nodes[n]['geometry'] = shapely.geometry.Point(G.nodes[n]['X'], G.nodes[n]['Y'])
for e in G.edges:
    edge = G.edges[e]
    edge['geometry'] = shapely.wkt.loads(edge['Wkt'])

env.FG = G

### Connect ports to simulation environment
We have a few different entities defined. The port contains cargo and a crane. The crane can be used to load and unload cargo from ships. The site objects (later to be extended with sluices, stopping areas, etc) are geojson features. 

In [5]:
ports = []
for site in config['sites']:
    port = dtv_backend.simple.Port(env, **site['properties'], **site)
    ports.append(port)

## Connect ships to simulation environment
The ships can also contain cargo. The ships can move over the graph. They are instances of the prototype ships from the Rijkswaterstaat ship dataset. You can have multiple copies of the same ship. All ships can work at the same time. 

In [6]:
ships = []
for ship in config['fleet']:
    kwargs = {}
    kwargs.update(ship)
    kwargs.update(ship['properties'])
    # the ship needs to know about the climate
    if 'climate' in config:
        kwargs['climate'] = config['climate']
    ship = dtv_backend.simple.Ship(env, **kwargs)
    ships.append(ship)

# Definition of functions

This section defined functions to select berth places on a given route from source to destination.

In [7]:
# relevant imports for functions
import datetime
import pandas as pd
import networkx as nx

import dtv_backend

In [8]:
def compute_path_length(graph, path, key='length_m'):
    """ aux fcn to compute distance of a path """
    total_distance = 0
    for e in zip(path[:-1], path[1:]):
        edge_distance = graph.edges[e][key]
        total_distance += edge_distance
    return total_distance

### Find berths on a route

In [9]:
class BerthFinder:
    """
    Class to find berths on a route.
    """
    def __init__(self, env, graph, src_node, dst_node, keyword='Berth', distance='length_m'):
        """
        Initialize with a graph, a path from source to destination, the keyword
        to identify berth nodes in the graph
        """
        self.env = env
        self.graph = graph
        self.src_node = src_node
        self.dst_node = dst_node
        self.path = self.find_path(src_node, dst_node)
        
        self.keyword = keyword
        self.distance = distance
    
    @property
    def time_now(self) -> datetime.datetime:
        """get the current time from the environment"""
        return datetime.datetime.fromtimestamp(self.env.now)
    
    def find_path(self, src, dst):
        """finds a path from src to dst on the given graph"""
        return dtv_backend.fis.shorted_path(self.graph, src, dst)
        
    def find_berths_near_route(self, max_distance=1000):
        """
        Looks for berth places within a given distance from a given path on a 
        graph. 
        """
        # capture berths in a set
        berths = set()
        
        # loop the nodes in the path
        for node in self.path:
            # generate a subgraph within given distance of the node
            new_graph = nx.generators.ego_graph(self.graph,
                                                node,
                                                radius=max_distance,
                                                distance=self.distance)
            # look for berths
            candidate_berths = [n for n in new_graph.nodes if n.startswith(self.keyword)]
            berths = berths.union(set(candidate_berths))
        
        return list(berths)

    def compute_eta_per_berth(self, berth_nodes, mean_speed):
        """
        Computes a speed-based estimated time of arrival for a given set of 
        berths on a route from source to destination, on a graph.
        """
        # define a set of candicate nodes
        candidate_nodes = berth_nodes + [self.dst_node]
        
        # compute the ETA of all candidate nodes from the current node
        rows = []
        for berth in candidate_nodes:
            # get the path from src_node to berth and compute distance and speed-based duration
            path_src_to_node = self.find_path(self.src_node, berth)
            distance_src_to_node = compute_path_length(self.graph, path_src_to_node)
            duration_src_to_node = distance_src_to_node / mean_speed
            
            # compute the distance from berth to destination
            path_node_to_dst = self.find_path(berth, self.dst_node)
            distance_node_to_dst = compute_path_length(self.graph, path_node_to_dst)
            
            # define a row to add to rows
            row = {'berth': berth,
                   'distance from src': distance_src_to_node,
                   'duration from src': duration_src_to_node,
                   'eta': self.time_now + datetime.timedelta(seconds=duration_src_to_node),
                   'distance to dst': distance_node_to_dst}
            
            # add the row
            rows.append(row)
        
        # convert to dataframe
        df = pd.DataFrame(rows)
        
        # compute the distance of the direct path from src to dst
        direct_path = self.find_path(self.src_node, self.dst_node)
        direct_distance = compute_path_length(self.graph, direct_path)
        
        # remove berths which take us away from the destination
        df = df.loc[df['distance to dst']<=direct_distance]
        
        return df
    
    def find_best_berth(self, df, max_timestamp):
        """
        Selects the 'best berth' based on the output of self.compute_eta_per_berth
        """
        df['feasible'] = df['eta'] <= max_timestamp
        
        # get the berth which minimizes the distance to the destination
        if sum(df['feasible']) == 0:
            # select the one in minimal time?
            next_berth = df.loc[df['eta'].idxmin(), 'berth']
        else:
            df_feas = df.loc[df['feasible']]
            next_berth = df_feas.loc[df_feas['distance to dst'].idxmin(), 'berth']
        
        return next_berth
    
    def next_berth(self, max_timestamp, mean_speed, max_distance):
        """return the next berth"""
        berths = self.find_berths_near_route(max_distance=max_distance)
        eta = berth_finder.compute_eta_per_berth(berth_nodes=berths,
                                                 mean_speed=mean_speed)
        berth = self.find_best_berth(df=eta,
                                     max_timestamp=max_timestamp)
        
        return berth

### Remove Berths which require additional lock passings

In [10]:
#TODO: add to class

### Remove Berths for which dimensions do not comply

In [11]:
#TODO: add to class

## Route from src to dst

In [12]:
# define nodes for source and destination
src = ports[0].node
dst = ports[1].node

## Find berths within given distance from route

In [13]:
berth_finder = BerthFinder(env, env.FG, src, dst)

max_timestamp = datetime.datetime(2022, 5, 5, 22, 0) #dummy
b = berth_finder.next_berth(max_timestamp,
                            mean_speed=3,
                            max_distance=1000)

In [14]:
b

'Berth229'